## Building graph using Nobel_prize JSON data

In [1]:
import json
import pandas as pd

file1 = '../data/Nobel_prize.json'
file2 = '../data/Nobel_laureate.json'
file3 = '../data/Nobel_country.json' # this file has simple json structure

In [2]:
with open(file1) as json_file:
    json_data1 = json.load(json_file)
# json_data

In [3]:
# from pandas.io.json import json_normalize

# # this is not practical for the this json data
# df = json_normalize(json_data1, 'prizes')
# df.head()

In [4]:
json_data1['prizes'][0]['laureates']

[{'id': '960',
  'firstname': 'Arthur',
  'surname': 'Ashkin',
  'motivation': '"for the optical tweezers and their application to biological systems"',
  'share': '2'},
 {'id': '961',
  'firstname': 'Gérard',
  'surname': 'Mourou',
  'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"',
  'share': '4'},
 {'id': '962',
  'firstname': 'Donna',
  'surname': 'Strickland',
  'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"',
  'share': '4'}]

In [5]:
with open(file2) as json_file:
    json_data2 = json.load(json_file)

In [6]:
json_data2['laureates'][100]

{'id': '103',
 'firstname': 'Ben Roy',
 'surname': 'Mottelson',
 'born': '1926-07-09',
 'died': '0000-00-00',
 'bornCountry': 'USA',
 'bornCountryCode': 'US',
 'bornCity': 'Chicago, IL',
 'gender': 'male',
 'prizes': [{'year': '1975',
   'category': 'physics',
   'share': '3',
   'motivation': '"for the discovery of the connection between collective motion and particle motion in atomic nuclei and the development of the theory of the structure of the atomic nucleus based on this connection"',
   'affiliations': [{'name': 'Nordita',
     'city': 'Copenhagen',
     'country': 'Denmark'}]}]}

In [7]:
# with open(file3) as json_file:
#     json_data3 = json.load(json_file)

# json_data3['countries']

In [8]:
import networkx as nx
from pprint import pprint
from graphgen import create_graph

In [9]:
nodes_mapper = {
    'nodes': [
        {
            'type': 'Affiliations',
            'path': '/prizes/affiliations',
            'key' : [
                {'name': 'name', 'raw': '/prizes/affiliations/name'}
            ],
            'attributes': [
                {'name': 'name',    'raw': '/prizes/affiliations/name'},
                {'name': 'city',    'raw': '/prizes/affiliations/city'},
                {'name': 'country', 'raw': '/prizes/affiliations/country'},
            ]
        },
        {
            'type': 'Prize',
            'path': '/prizes',
            'key' : [
                {'name': 'category', 'raw': '/prizes/category'}
            ],
            'attributes': [
                {'name': 'category', 'raw': '/prizes/category'},
            ]
        }
    ]
}
edges_mapper = {
    'edges': [
        {
            'type': 'Awarded',
            'from': {
                'type': 'Affiliations',
                'path': '/prizes/affiliations',
                'key' : [
                    {'name': 'name', 'raw': '/prizes/affiliations/name'}
                ]
            },
            'to'  : {
                'type': 'Prize',
                'path': '/prizes',
                'key' : [
                    {'name': 'category', 'raw': '/prizes/category'}
                ]
            },
            'attributes': [
                {'name': 'year', 'raw': '/prizes/year'}, 
            ]
        }
    ]
}

In [10]:
# def extract_node_attrs_from_json(jdata, type_path, attr_list):
# #     print('>>> looking for:', type_path)
# #     print('>>> looking for attrs:', attr_list)
#     out = []

#     # make sure our type_path end with '/'
#     if type_path[-1] != '/':
#         type_path += '/' 
    
    
#     def extract_data(jdata, cur_path = '/', cur_obj = None):
#         if type(jdata) is dict:            
#             if cur_path == type_path:
# #                 print('<<< MATCHED_TYPE >>>')
# #                 print('@path:', cur_path)
#                 obj = {}
#                 for a in jdata:
#                     extract_data(jdata[a], cur_path + a + '/', obj)
# #                 print('>>> got obj', obj)
#                 out.append(obj)
#             else:
#                 for a in jdata:
#                     extract_data(jdata[a], cur_path + a + '/')
#         elif type(jdata) is list:
#             for a in jdata:
#                 extract_data(a, cur_path)
#         else:
# #             print('cur_path: {} - type_path: {}'.format(cur_path, type_path))
#             if cur_obj != None and cur_path in attr_list:
# #                 print('<<< MATCHED_ATTR >>>')
#                 cur_obj[cur_path] = jdata
    
#     extract_data(jdata)
#     return out
                
    

In [11]:
# def extract_edge_attrs_from_json(jdata, src_type_path, dst_type_path, attr_list):
# #     print('>>> looking for:', type_path)
# #     print('>>> looking for attrs:', attr_dict)
#     out = []

#     # make sure our type_path end with '/'
#     if src_type_path[-1] != '/':
#         src_type_path += '/' 
    
#     if dst_type_path[-1] != '/':
#         dst_type_path += '/' 
        
#     def extract_data(jdata, cur_path = '/', cur_obj = None, got_from = False, got_to = False):
#         if type(jdata) is dict:            
#             if cur_path == src_type_path or cur_path == dst_type_path:
#                 if cur_path == src_type_path:
#                     got_from = True
#                 else:
#                     got_to = True
# #                 print('<<< MATCHED_TYPE >>>')
# #                 print('@path:', cur_path)
#                 obj = {}
#                 if cur_obj != None: # we're still collecting
#                     obj = cur_obj
#                 for a in jdata:
#                     extract_data(jdata[a], cur_path + a + '/', obj, got_from, got_to)
# #                     print('>>> got edge data:', obj)
# #                 print('got_from:', got_from, ' - got_to:', got_to)
#                 if got_from and got_to:
#                     out.append(obj)
#                     got_from = got_to = False
#             else:
#                 for a in jdata:
#                     extract_data(jdata[a], cur_path + a + '/', cur_obj, got_from, got_to)
#         elif type(jdata) is list:
#             for a in jdata:
#                 extract_data(a, cur_path, cur_obj, got_from, got_to)
#         else:
# #             print('cur_path: {} - src_type_path: {} - dst_type_path: {}'.format(cur_path, 
# #                                                                                 src_type_path, dst_type_path))
#             if cur_obj != None and cur_path in attr_list:
# #                 print('<<< MATCHED_ATTR >>>')
#                 cur_obj[cur_path] = jdata

#     extract_data(jdata)
#     return out
                

In [12]:
# def create_graph_nodes_from_json(graph, graph_mapper, data_provider, update = True):
#     '''
    
#     params:
#         graph: fully constructed graph object to add new nodes and edges to it.
#         graph_mapper: dictionary describing the type of object to extract
#         data_provider: json_data
        
#     return:
#         constructured "graph_type" graph object based on the provided source data and according to 
#         the mapper schema description.
#     '''

#     assert (graph != None),"Graph object wasn't constructed correctly"
#     # TBD... assert (isinstance(data_provider, pd.DataFrame)),"The data provider should be a pandas DataFrame"
    
#     # get list of node types and edge types
#     node_types = []
#     edge_types = []

#     if 'nodes' in graph_mapper.keys():
#         node_types = graph_mapper['nodes']
#     if 'edges' in graph_mapper.keys():
#         edge_types = graph_mapper['edges']

#     raw_data = data_provider
    
# #     print(node_types)
# #     print(edge_types)
    
#     for node_type in node_types:
#         # TBD... assert check_attributes(node_type, raw_data, node_type['attributes'])
       
#         # TBD: Need to support multiple keys. For now we'll only have a single key for each record 
# #         print('node_type to process:', node_type)
#         node_key = node_type['key'][0]
#         key_name = node_key['name']
#         key_raw_name = node_key['raw']

#         attr_dict = {}
#         for a in node_type['attributes']:
#             raw_attr = a['raw']
#             if raw_attr[-1] != '/':
#                 raw_attr += '/'
#             attr_dict[a['name']] = raw_attr
        
#         attr = dict()
#         count = 0
#         node_type_name = node_type['type']
#         node_type_path = node_type['path']
#         if node_type_path[-1] != '/':
#             node_type_path += '/' 
 
#         # construct attribute mapping between raw_attrib_name -> attrib_name
#         lookup_attr_list = []
    
#         for k, v in attr_dict.items():
#             if v[-1] != '/':
#                 v += '/'
#             lookup_attr_list.append(v)
        
#         # make sure we have the key_raw_name in the list of attributes
#         if key_raw_name[-1] != '/':
#             key_raw_name += '/'
#         if key_raw_name not in lookup_attr_list:
#             lookup_attr_list.append(key_raw_name_path)
# #         print('lookup_attr_list:', lookup_attr_list)
        
#         # iterate and collect.  
#         for j in raw_data:
# #             print('json>> ', j)
#             jelem = extract_node_attrs_from_json(j, node_type_path, lookup_attr_list)
#             if len(jelem) > 0:
#                 for e in jelem:
# #                     print('{} - type_found: {} - attr: {}'.format(count, node_type_name, e))
#                     key_value = e[key_raw_name] if key_raw_name in e else 'UNKNOWN_'+str(count)
#                     node_id = '{}_{}'.format(node_type_name, key_value)
#                     if not update and graph.has_node(node_id):
# #                         print('graph has node', node_id)
#                         continue

#                     attr['_type_'] = node_type_name
#                     for k,v in attr_dict.items():
#                         attr[k] = e[v] if v in e else ''
# #                     print('>> adding node: ', node_id)
#                     graph.add_node(node_id, **attr)
#                     count += 1
        
#         print('type: {} - {}'.format(node_type_path, count))
        
#     return graph

In [13]:
# def create_graph_edges_from_json(graph, graph_mapper, data_provider, update = True):
#     '''
    
#     params:
#         graph: fully constructed graph object to add new nodes and edges to it.
#         graph_mapper: dictionary describing the type of object to extract
#         data_provider: json_data
        
#     return:
#         constructured "graph_type" graph object based on the provided source data and according to 
#         the mapper schema description.
#     '''

#     assert (graph != None),"Graph object wasn't constructed correctly"
#     # TBD... assert (isinstance(data_provider, pd.DataFrame)),"The data provider should be a pandas DataFrame"
    
#     # get list of edge types and edge types
#     edge_types = []

#     if 'edges' in graph_mapper.keys():
#         edge_types = graph_mapper['edges']

#     raw_data = data_provider
    
# #     print(edge_types)
#     for edge_type in edge_types:
        
#         # TBD... assert check_attributes(edge_type, raw_data, edge_type['attributes'])

#         # TBD: Need to support multiple keys. For now we'll only have a single key for each record 
#         edge_type_name = edge_type['type']

#         # source node metadata
#         src_type_name = edge_type['from']['type']
#         src_type_path = edge_type['from']['path']
#         if src_type_path[-1] != '/':
#             src_type_path += '/' 

#         src_key = edge_type['from']['key'][0]
#         src_key_name = src_key['name']
#         src_key_raw_name = src_key['raw']
#         if src_key_raw_name[-1] != '/':
#             src_key_raw_name += '/'
        
#         # destination node metadata
#         dst_type_name = edge_type['to']['type']
#         dst_type_path = edge_type['to']['path']
#         if dst_type_path[-1] != '/':
#             dst_type_path += '/' 

#         dst_key = edge_type['to']['key'][0]
#         dst_key_name = dst_key['name']
#         dst_key_raw_name = dst_key['raw']
#         if dst_key_raw_name[-1] != '/':
#             dst_key_raw_name += '/'
    

#         attr_dict = {}
#         for a in edge_type['attributes']:
#             v = a['raw']
#             if v[-1] != '/':
#                 v += '/'
#             attr_dict[a['name']] = v
            
#         attr = dict()
#         count = 0
 
#         # construct attribute mapping between raw_attrib_name_path -> attrib_name
#         lookup_attr_list = []
    
#         # TBD: review this code, not sure if it's necessary
#         for k, v in attr_dict.items():
#             lookup_attr_list.append(v)


#         # make sure we have the src_key_raw_name in the list of attributes
#         if src_key_raw_name not in lookup_attr_list:
#             lookup_attr_list.append(src_key_raw_name)
            
#         # make sure we have the dst_key_raw_name in the list of attributes
#         if dst_key_raw_name not in lookup_attr_list:
#             lookup_attr_list.append(dst_key_raw_name)

# #         print('lookup_attr_list:', lookup_attr_list)

        
#         # iterate and collect.  
#         for j in raw_data:
# #             print('json>> ', j)
#             jelem = extract_edge_attrs_from_json(j, src_type_path, dst_type_path, lookup_attr_list)
#             if len(jelem) > 0:
#                 for e in jelem:
# #                     print('{} - src: {} - dest: {} - attr: {}'.format(count, src_type_name, dst_type_name, e))
#                     src_key_value = e[src_key_raw_name] if src_key_raw_name in e else 'UNKNOWN_'+str(count)
#                     dst_key_value = e[dst_key_raw_name] if dst_key_raw_name in e else 'UNKNOWN_'+str(count)
        
#                     attr['_type_'] = edge_type_name
#                     for k,v in attr_dict.items():
#                         attr[k] = e[v] if v in e else ''
#                     from_id = '{}_{}'.format(src_type_name, src_key_value)
#                     to_id = '{}_{}'.format(dst_type_name, dst_key_value)
# #                     print('adding edge from: {} -> to: {}, attr: {}'.format(from_id, to_id, attr))
#                     graph.add_edge(from_id, to_id, **attr)
                    
#                     count += 1
        
#         print('type: {} -> {} - {}'.format(src_type_path, dst_type_path, count))
       
#     return graph

In [14]:
g = nx.MultiDiGraph()

g = create_graph(g, graph_mapper = nodes_mapper, 
                 data_provider = json_data2['laureates'])

g = create_graph(g, graph_mapper = edges_mapper, 
                 data_provider = json_data2['laureates'])

type: /prizes/affiliations/ - 753
type: /prizes/ - 941
type: /prizes/affiliations/ -> /prizes/ - 753


In [16]:
type(g)

networkx.classes.multidigraph.MultiDiGraph

In [17]:
nx.number_of_nodes(g)

341

In [18]:
nx.number_of_edges(g)

753

In [19]:
pprint(json_data2['laureates'][216])

{'born': '1907-10-02',
 'bornCity': 'Glasgow',
 'bornCountry': 'Scotland',
 'bornCountryCode': 'GB',
 'died': '1997-01-10',
 'diedCity': 'Cambridge',
 'diedCountry': 'United Kingdom',
 'diedCountryCode': 'GB',
 'firstname': 'Lord (Alexander R.)',
 'gender': 'male',
 'id': '221',
 'prizes': [{'affiliations': [{'city': 'Cambridge',
                               'country': 'United Kingdom',
                               'name': 'University of Cambridge'}],
             'category': 'chemistry',
             'motivation': '"for his work on nucleotides and nucleotide '
                           'co-enzymes"',
             'share': '1',
             'year': '1957'}],
 'surname': 'Todd'}


In [20]:
print(g.node['Affiliations_University of Cambridge'])

{'_type_': 'Affiliations', 'name': 'University of Cambridge', 'city': 'Cambridge', 'country': 'United Kingdom'}


In [21]:
print(g.node['Prize_chemistry'])
print(g.node['Prize_physics'])

{'_type_': 'Prize', 'category': 'chemistry'}
{'_type_': 'Prize', 'category': 'physics'}


In [22]:
pprint(g.get_edge_data('Affiliations_University of Cambridge', 'Prize_chemistry'))

{0: {'_type_': 'Awarded', 'year': '1922'},
 1: {'_type_': 'Awarded', 'year': '1957'},
 2: {'_type_': 'Awarded', 'year': '1958'}}


In [ ]:
pprint(g.get_edge_data('Prize_chemistry', 'Affiliations_University of Cambridge'))